# Fine-tuning BERT (base or large) on a Question-Answering task by using the library adapter-transformers (notebook version) and its Fusion mode

- **Credit**: [Hugging Face](https://huggingface.co/) and [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)
- **Author**: [Pierre GUILLOU](https://www.linkedin.com/in/pierreguillou/)
- **Date**: 08/02/2021
- **Blog post**: []()
- **Link to the folder in github with this notebook and all necessary scripts**: [question-answering with adapters](https://github.com/piegu/language-models/tree/master/adapters/question-answering/)

## 1. Context

### Objective

The objective here is to **fine-tune a Masked Language Model (MLM) like BERT (base or large) for a QA task by training adapters (library [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)) on its Fusion mode, not the embeddings and transformers layers of the MLM model**, and to compare results with BERT model fully fine-tune for the same task.

The interest is obvious: if you need models for different NLP tasks, instead of fine-tuning and storing one model by NLP task, **you store only one MLM model and the trained tasks adapters which sizes are between 1% (QA adapter) and 8% (Lang+QA adapters) of the MLM model one** (it depends of the choosen adapter configuration). **With the fusion mode, there is one more adapter by layer (the fusion adapter) that raises the number of adapter parameters from 23.5% (QA+NER adapters) to 31% (LANG+QA+NER adapters).** More, the loading of these adapters in production is very easy.

The difference here with our others notebooks about Question-Answering (QA) with Adapters (see our folder [question-answering](https://github.com/piegu/language-models/tree/master/adapters/question-answering) in github) is **the use of the Fusion mode**: instead of training only a task adapter on QA (with or without a lang adapter), we've trained a fusion adapter in QA that will make a weighted composite of the parameters of the adapters added to our model. **The idea here is not just to kepp all knowledge from other tasks, but to compose with it.**

![Adapter Fusion architecture](images/adapterfusion1.png)

### Content

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will use the library [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers) and see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](images/question_answering_adapter.png)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

### History and Credit

This notebook is an adaptation of the following notebooks and scripts for **fine-tuning a (transformer) Masked Language Model (MLM) like BERT (base or large) on the QA task with any QA dataset** (we use here the [Portuguese Squad 1.1 dataset](https://forum.ailab.unb.br/t/datasets-em-portugues/251/4)):
- **from [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers)** | notebook [03_Adapter_Fusion.ipynb](https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/03_Adapter_Fusion.ipynb), notebook [04_Cross_Lingual_Transfer.ipynb](https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/04_Cross_Lingual_Transfer.ipynb) and script [run_qa.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/question-answering/run_qa.py) (this script was adapted from the script [run_qa.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py) of HF)
- **from [transformers](https://github.com/huggingface/transformers) of Hugging Face** | notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb) and script [run_qa.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py) 

In order to speed up the fine-tuning of the model on only one GPU, the library [DeepSpeed](https://www.deepspeed.ai/) could be used by applying the configuration provided by HF in the notebook [transformers + deepspeed CLI](https://github.com/stas00/porting/blob/master/transformers/deepspeed/DeepSpeed_on_colab_CLI.ipynb) but as the library adapter-transformers is not synchronized with the last version of the library transformers of HF, we keep that option for the future.

### Major changes from original notebooks and scripts

The script [run_qa.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/question-answering/run_qa.py) allows to evaluate the model performance against f1 metric at the end of each epoch, and not against validation loss as done in the HF notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb). This is very important as we consider the metric when selecting a model, not the loss. Therefore, we decided to launch this script inside the notebook [question_answering_adapter_script.ipynb](https://github.com/piegu/language-models/blob/master/adapters/question-answering/question_answering_adapter_script.ipynb) (by simulating terminal command line) instead of running code in cells as done in the HF notebook. 

However, to provide an HF-like notebook, we also updated the HF notebook [question_answering.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb) on this notebook ([question_answering_adapter.ipynb](https://github.com/piegu/language-models/blob/master/adapters/question-answering/question_answering_adapter.ipynb)), but without the f1 metric as the evaluation metric during model training (ie, the f1 metric is used only after the model training).

More, we updated the script [run_qa.py](https://github.com/Adapter-Hub/adapter-transformers/blob/master/examples/question-answering/run_qa.py) to [run_qa_adapter.py](https://github.com/piegu/language-models/blob/master/adapters/question-answering/run_qa_adapter.py) with the following changes:
- **EarlyStopping** by selecting the model with the highest eval f1 (patience of 3 before ending the training)
- **MAD-X 2.0** that allows not to train adapters in the last transformer layer (read page 6 of [UNKs Everywhere: Adapting Multilingual Language Models to New Scripts](https://arxiv.org/pdf/2012.15562.pdf))
- **Stack method** for the lang and task adapters when a lang adapter is loaded ([doc](https://docs.adapterhub.ml/adapter_composition.html?highlight=stack#stack))
- **Fusion method** of 2 task adapters (see [AdapterFusion: Non-Destructive Task Composition for Transfer Learning](https://arxiv.org/pdf/2005.00247.pdf)).
- **Houlsby MHA last layer** that allows no to train the task adapter after the Feed Fordward but only after the MHA (Multi-Head Attention) in the last layer for the Houlsby configuration

## 2. Installation

In [1]:
import pathlib
from pathlib import Path

#root path
root = Path.cwd()

In [2]:
import pickle
import pandas as pd

In [3]:
import sys; print('python:',sys.version)

import torch; print('Pytorch:',torch.__version__)

import transformers; print('adapter-transformers:',transformers.__version__)
import transformers; print('HF transformers:',transformers.__hf_version__)
import tokenizers; print('tokenizers:',tokenizers.__version__)
import datasets; print('datasets:',datasets.__version__)

# import deepspeed; print('deepspeed:',deepspeed.__version__)

# Versions used in the virtuel environment of this notebook:

# python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
# [GCC 7.5.0]
# Pytorch: 1.9.0
# adapter-transformers: 2.1.1
# HF transformers: 4.8.2
# tokenizers: 0.10.3
# datasets: 1.9.0

python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]
Pytorch: 1.9.0
adapter-transformers: 2.1.1
HF transformers: 4.8.2
tokenizers: 0.10.3
datasets: 1.9.0


## 3. Model & dataset

In [4]:
# Select a MLM BERT base or large in the dataset language
# model_checkpoint = "neuralmind/bert-base-portuguese-cased"
model_checkpoint = "neuralmind/bert-large-portuguese-cased"
model_checkpoint_original = "neuralmind/bert-large-portuguese-cased"
model_checkpoint_original_local = root.parent/'neuralmind-bert-large-portuguese-cased-lenerbr/mlm_base'

# SQuAD 1.1 in Portuguese
dataset_name = "squad11pt"

# datasets of task adapters already trained
dataset_name_ad1 = "squad11pt"
dataset_name_ad2 = "lenerbr"

# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False # If true, some of the examples do not have an answer.

## 4. Main hyperparameters

In [5]:
task = "qa_fusion"

# tasks of task adapters already trained
task_ad1 = "qa"
task_ad2 = "ner"

In [6]:
# training arguments
batch_size = 16
gradient_accumulation_steps = 2

learning_rate = 3e-5
num_train_epochs = 5.
early_stopping_patience = 3

weight_decay = 0.01

adam_epsilon = 1e-6

fp16 = True
ds = False # If True, we use DeepSpeed
if ds:
    zero2 = True

# best model
load_best_model_at_end = True 
metric_for_best_model = "loss"
greater_is_better = False

In [7]:
# train adapter
train_adapter = True # we want to train an adapter
load_adapter = None # we do not upload an existing adapter 

# lang adapter
with_adapters_mlm = True # if False, we do not upload an existing lang adapter

if with_adapters_mlm:
    adapter_composition = "stack" # we will stack the lang and task adapters
else:
    adapter_composition = None

# if True, do not put adapter in the last transformer layer (Pfeiffer configuration)
madx2 = True

# if True, put only an adapter after the MHA but not after the FF in the last layer (Houlsby configuration)
houlsby_MHA_lastlayer = False
if madx2:
    houlsby_MHA_lastlayer = False

## 5. Configuration

### GPU

In [8]:
# gpu
n_gpu = 1 # train on just one GPU
gpu = 1 # select the GPU

In [9]:
# Select GPU 0
import os
os.environ['MASTER_ADDR'] = 'localhost'
if gpu == 0:
    os.environ['MASTER_PORT'] = '9993' # modify if RuntimeError: Address already in use # GPU 0
elif gpu == 1:
    os.environ['MASTER_PORT'] = '9995'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = str(gpu)
os.environ['WORLD_SIZE'] = "1"

### DeepSpeed

In [10]:
if ds:
    if zero2:
        ds_config = 'ds_config_zero2.json'
    elif zero3:
        ds_config = 'ds_config_zero3.json'       

Let's setup the `DeepSpeed` configuration.

To use a LR Linear Decay after warmup as scheduler (equivalent of the one by default in the HF Trainer), we changed `WarmupLR` to `WarmupDecayLR` in the DeepSpeed configuration file, and kept a copy of the scheduler initial code here:

````
# the lr stays constant after the warmup (this is not equivalent to teh default scheduler of HF which is Linear)
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
````

#### ZeRO-2

source: https://huggingface.co/transformers/master/main_classes/deepspeed.html#zero-2-example

In [11]:
%%bash

cat <<'EOT' > ds_config_zero2.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
         "type": "WarmupDecayLR",
         "params": {
             "last_batch_iteration": -1,
             "total_num_steps": "auto",
             "warmup_min_lr": "auto",
             "warmup_max_lr": "auto",
             "warmup_num_steps": "auto"
         }
     },

    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "allgather_partitions": true,
        "allgather_bucket_size": 2e8,
        "overlap_comm": true,
        "reduce_scatter": true,
        "reduce_bucket_size": 2e8,
        "contiguous_gradients": true
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
EOT

#### ZeRO-3

In [12]:
%%bash

cat <<'EOT' > ds_config_zero3.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
         "type": "WarmupDecayLR",
         "params": {
             "last_batch_iteration": -1,
             "total_num_steps": "auto",
             "warmup_min_lr": "auto",
             "warmup_max_lr": "auto",
             "warmup_num_steps": "auto"
         }
     },

    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_fp16_weights_on_model_save": true
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
EOT

### Adapters config

#### Task adapter

In [13]:
# task adapter config
adapter_config_name = "pfeiffer" # houlsby is possible, too
if adapter_config_name == "pfeiffer":
    adapter_non_linearity = 'gelu' # relu is possible, too
elif adapter_config_name == "houlsby":
    adapter_non_linearity = 'swish'
adapter_reduction_factor = 16
language = 'pt' # pt = Portuguese

#### Lang adapter

In [14]:
if with_adapters_mlm:
    
    # hyperparameters used for fine-tuning the MLM with lang adapter
    learning_rate_mlm = 1e-4
    batch_size_mlm = 16
    gradient_accumulation_steps_mlm = 1
    adam_epsilon_mlm = 1e-4
    num_train_epoch_mlm = 100.
    early_stopping_patience_mlm = 10
    madx2_mlm = madx2
    houlsby_MHA_lastlayer_mlm = houlsby_MHA_lastlayer
    ds_mlm = False
    fp16_mlm = True
    load_best_model_at_end_mlm = True
    metric_for_best_model_mlm = "loss"
    adapter_config_mlm = adapter_config_name + '+inv'

    # path to lang adapter
    outputs_mlm = model_checkpoint.replace('/','-') + '_' + dataset_name + '/' + 'mlm' + '/' \
    + 'lr' + str(learning_rate_mlm) \
    + '_bs' + str(batch_size_mlm) \
    + '_GAS' + str(gradient_accumulation_steps_mlm) \
    + '_eps' + str(adam_epsilon_mlm) \
    + '_epochs' + str(num_train_epoch_mlm) \
    + '_patience' + str(early_stopping_patience_mlm) \
    + '_madx2' + str(madx2_mlm) \
    + '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer_mlm) \
    + '_ds' + str(ds_mlm) \
    + '_fp16' + str(fp16_mlm) \
    + '_best' + str(load_best_model_at_end_mlm) \
    + '_metric' + str(metric_for_best_model_mlm) \
    + '_adapterconfig' + str(adapter_config_mlm)

    path_to_outputs_mlm = root/'outputs'/outputs_mlm
    
    # Config of the lang adapter
    lang_adapter_path = path_to_outputs_mlm/'adapters-mlm/'
    
    load_lang_adapter = str(lang_adapter_path)
    lang_adapter_config = str(lang_adapter_path) + "/adapter_config.json"
    if adapter_config_mlm == "pfeiffer+inv":
        lang_adapter_non_linearity = 'gelu' # relu is possible, too
    elif adapter_config_mlm == "houlsby+inv":
        lang_adapter_non_linearity = 'swish'
    lang_adapter_reduction_factor = 2
    language_mlm = language
    
else:
    type_lang_adapter = None

    load_lang_adapter = None
    lang_adapter_config = None
    lang_adapter_non_linearity = None
    lang_adapter_reduction_factor = None
    
    language_mlm = None 

#### QA Adapter Arguments

In [15]:
# folder for training outputs

outputs_ad1 = model_checkpoint.replace('/','-') + '_' + dataset_name_ad1

if with_adapters_mlm:
    outputs_ad1 = outputs_ad1 + '/' + 'mlm_' + str(task_ad1) + '_AdCompo' + str(adapter_composition) + '/'
else:
    outputs_ad1 = outputs_ad1 + '/' + str(task_ad1) + '/'

# with LANG adapter
if with_adapters_mlm:  
    learning_rate_ad1 = 1e-4
    batch_size_ad1 = 8
    gradient_accumulation_steps_ad1 = 2
    adam_epsilon_ad1 = 1e-7
    num_train_epochs_ad1 = 15.0
    early_stopping_patience_ad1 = 5
    with_adapters_mlm_ad1 = with_adapters_mlm
    madx2_ad1 = madx2
    houlsby_MHA_lastlayer_ad1 = houlsby_MHA_lastlayer
    ds_ad1 = False
    fp16_ad1 = True
    load_best_model_at_end_ad1 = load_best_model_at_end
    metric_for_best_model_ad1 = metric_for_best_model
    adapter_config_name_ad1 = adapter_config_name.replace('+inv','')
# without LANG adapter
else:
    learning_rate_ad1 = 1e-4
    batch_size_ad1 = 16
    gradient_accumulation_steps_ad1 = 1
    adam_epsilon_ad1 = 1e-8
    num_train_epochs_ad1 = 15.0
    early_stopping_patience_ad1 = 5
    with_adapters_mlm_ad1 = with_adapters_mlm
    madx2_ad1 = madx2
    houlsby_MHA_lastlayer_ad1 = houlsby_MHA_lastlayer
    ds_ad1 = False
    fp16_ad1 = True
    load_best_model_at_end_ad1 = load_best_model_at_end
    metric_for_best_model_ad1 = 'eval_loss'
    adapter_config_name_ad1 = adapter_config_name.replace('+inv','')

outputs_ad1 = outputs_ad1 \
+ 'lr' + str(learning_rate_ad1) \
+ '_bs' + str(batch_size_ad1) \
+ '_GAS' + str(gradient_accumulation_steps_ad1) \
+ '_eps' + str(adam_epsilon_ad1) \
+ '_epochs' + str(num_train_epochs_ad1) \
+ '_patience' + str(early_stopping_patience_ad1) \
+ '_wamlm' + str(with_adapters_mlm_ad1) \
+ '_madx2' + str(madx2_ad1) \
+ '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer_ad1) \
+ '_ds' + str(ds_ad1) \
+ '_fp16' + str(fp16_ad1) \
+ '_best' + str(load_best_model_at_end_ad1) \
+ '_metric' + str(metric_for_best_model_ad1) \
+ '_adapterconfig' + str(adapter_config_name_ad1)

# path to outputs
path_to_outputs_ad1 = root/'outputs'/outputs_ad1

path_to_save_adapter_ad1 = path_to_outputs_ad1/'adapters-qa'

load_qa_adapter = str(path_to_save_adapter_ad1)
qa_adapter_config = str(path_to_save_adapter_ad1) + "/adapter_config.json"
qa_adapter = "pfeiffer"
qa_adapter_non_linearity = 'gelu'
qa_adapter_reduction_factor = 16

#### NER Adapter Arguments

In [16]:
# folder for training outputs

outputs_ad2 = model_checkpoint.replace('/','-') + '_' + dataset_name_ad2 + '/' + str(task_ad2) + '/'
    
learning_rate_ad2 = 1e-4
batch_size_ad2 = 4
gradient_accumulation_steps_ad2 = 2
adam_epsilon_ad2 = 1e-4
num_train_epochs_ad2 = 30.0
early_stopping_patience_ad2 = 10
with_adapters_mlm_ad2 = False
madx2_ad2 = madx2
houlsby_MHA_lastlayer_ad2 = houlsby_MHA_lastlayer
ds_ad2 = False
fp16_ad2 = True
load_best_model_at_end_ad2 = load_best_model_at_end
metric_for_best_model_ad2 = 'f1'
weighted_loss_ad2 = True
if not weighted_loss_ad2:
    c_ad2 = 0.0
else:
    c_ad2 = 0.5
next_sentence_ad2 = True
adapter_config_name_ad2 = adapter_config_name.replace('+inv','')

outputs_ad2 = outputs_ad2 \
+ 'lr' + str(learning_rate_ad2) \
+ '_bs' + str(batch_size_ad2) \
+ '_GAS' + str(gradient_accumulation_steps_ad2) \
+ '_eps' + str(adam_epsilon_ad2) \
+ '_epochs' + str(num_train_epochs_ad2) \
+ '_patience' + str(early_stopping_patience_ad2) \
+ '_wamlm' + str(with_adapters_mlm_ad2) \
+ '_madx2' + str(madx2_ad2) \
+ '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer_ad2) \
+ '_ds' + str(ds_ad2) \
+ '_fp16' + str(fp16_ad2) \
+ '_best' + str(load_best_model_at_end_ad2) \
+ '_metric' + str(metric_for_best_model_ad2) \
+ '_weightedloss' + str(weighted_loss_ad2) \
+ '_c' + str(c_ad2) \
+ '_ns' + str(next_sentence_ad2) \
+ '_adapterconfig' + str(adapter_config_name_ad2)

# path to outputs
path_to_outputs_ad2 = root/'outputs'/outputs_ad2

path_to_save_adapter_ad2 = path_to_outputs_ad2/'adapters-ner'

load_ner_adapter = str(path_to_save_adapter_ad2)
ner_adapter_config = str(path_to_save_adapter_ad2) + "/adapter_config.json"
ner_adapter = "pfeiffer"
ner_adapter_non_linearity = 'gelu'
ner_adapter_reduction_factor = 16

### Training arguments of the HF trainer

In [18]:
# setup the training argument
do_train = True # False
do_eval = True 

# if you want to test the trainer, set up the following variables
max_train_samples = 200 # None
max_eval_samples = 50 # None

# epochs, bs, GA
evaluation_strategy = "epoch" # no

# fp16
fp16_opt_level = 'O1'
fp16_backend = "auto"
fp16_full_eval = False

# optimizer (AdamW)
weight_decay = 0.01 # 0.0
adam_beta1 = 0.9
adam_beta2 = 0.999

# scheduler
lr_scheduler_type = 'linear'
warmup_ratio = 0.0
warmup_steps = 0

# logs
logging_strategy = "steps"
logging_first_step = True # False
logging_steps = 500     # if strategy = "steps"
eval_steps = logging_steps # logging_steps

# checkpoints
save_strategy = "epoch" # steps
save_steps = 500 # if save_strategy = "steps"
save_total_limit = 1 # None

# no cuda, seed
no_cuda = False
seed = 42

# bar
disable_tqdm = False # True
remove_unused_columns = True

In [19]:
# folder for training outputs

outputs = model_checkpoint.replace('/','-') + '_' + dataset_name

if with_adapters_mlm:
    outputs = outputs + '/' + 'mlm_' + str(task) + '_AdCompo' + str(adapter_composition) + '/'
else:
    outputs = outputs + '/' + str(task) + '/'

outputs = outputs \
+ 'lr' + str(learning_rate) \
+ '_bs' + str(batch_size) \
+ '_GAS' + str(gradient_accumulation_steps) \
+ '_eps' + str(adam_epsilon) \
+ '_epochs' + str(num_train_epochs) \
+ '_patience' + str(early_stopping_patience) \
+ '_wamlm' + str(with_adapters_mlm) \
+ '_madx2' + str(madx2) \
+ '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer) \
+ '_ds' + str(ds) \
+ '_fp16' + str(fp16) \
+ '_best' + str(load_best_model_at_end) \
+ '_metric' + str(metric_for_best_model) \
+ '_adapterconfig' + str(adapter_config_name) \
+ '_ad1' + str(task_ad1) + dataset_name_ad1 \
+ '_ad2' + str(task_ad2) + dataset_name_ad2

# path to outputs
path_to_outputs = root/'outputs'/outputs

# subfolder for model outputs
output_dir = path_to_outputs/'output_dir' 
overwrite_output_dir = True # False

# logs
logging_dir = path_to_outputs/'logging_dir'

## 6. Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [21]:
# %%time
# if dataset_name == "squad11pt":
    
#     # create dataset folder 
#     path_to_dataset = root.parent/'data'/dataset_name
#     path_to_dataset.mkdir(parents=True, exist_ok=True) 

#     # Get dataset SQUAD in Portuguese
#     %cd {path_to_dataset}
#     !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn" -O squad-pt.tar.gz && rm -rf /tmp/cookies.txt

#     # unzip 
#     !tar -xvf squad-pt.tar.gz

#     # Get the train and validation json file in the HF script format 
#     # inspiration: file squad.py at https://github.com/huggingface/datasets/tree/master/datasets/squad

#     import json 
#     files = ['squad-train-v1.1.json','squad-dev-v1.1.json']

#     for file in files:

#         # Opening JSON file & returns JSON object as a dictionary 
#         f = open(file, encoding="utf-8") 
#         data = json.load(f) 

#         # Iterating through the json list 
#         entry_list = list()
#         id_list = list()

#         for row in data['data']: 
#             title = row['title']

#             for paragraph in row['paragraphs']:
#                 context = paragraph['context']

#                 for qa in paragraph['qas']:
#                     entry = {}

#                     qa_id = qa['id']
#                     question = qa['question']
#                     answers = qa['answers']

#                     entry['id'] = qa_id
#                     entry['title'] = title.strip()
#                     entry['context'] = context.strip()
#                     entry['question'] = question.strip()

#                     answer_starts = [answer["answer_start"] for answer in answers]
#                     answer_texts = [answer["text"].strip() for answer in answers]
#                     entry['answers'] = {}
#                     entry['answers']['answer_start'] = answer_starts
#                     entry['answers']['text'] = answer_texts

#                     entry_list.append(entry)

#         reverse_entry_list = entry_list[::-1]

#         # for entries with same id, keep only last one (corrected texts by the group Deep Learning Brasil)
#         unique_ids_list = list()
#         unique_entry_list = list()
#         for entry in reverse_entry_list:
#             qa_id = entry['id']
#             if qa_id not in unique_ids_list:
#                 unique_ids_list.append(qa_id)
#                 unique_entry_list.append(entry)

#         # Closing file 
#         f.close() 

#         new_dict = {}
#         new_dict['data'] = unique_entry_list

#         file_name = 'pt_' + str(file)
#         with open(file_name, 'w') as json_file:
#             json.dump(new_dict, json_file)
            
# %cd {root}

In [22]:
# from datasets import load_dataset
# datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [23]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
from datasets import load_dataset, load_metric

if dataset_name == "squad11pt":
    
    # dataset folder 
    root = Path.cwd()
    path_to_dataset = root.parent/'data'/dataset_name
    
    # paths to files
    train_file = str(path_to_dataset/'pt_squad-train-v1.1.json')
    validation_file = str(path_to_dataset/'pt_squad-dev-v1.1.json')
    
    datasets = load_dataset('json', 
                            data_files={'train': train_file, \
                                        'validation': validation_file, \
                                       }, 
                            field='data')

In [25]:
datasets["train"][10]

{'id': '5735c47ae853931400426b64',
 'title': 'Kathmandu',
 'context': 'A maioria das cozinhas encontradas em Katmandu não é vegetariana. No entanto, a prática do vegetarianismo não é incomum, e a culinária vegetariana pode ser encontrada em toda a cidade. O consumo de carne bovina é muito incomum e considerado tabu em muitos lugares. Buff (carne de búfalo Marinho) é muito comum. Há uma forte tradição de consumo de buffs em Katmandu, especialmente entre Newars, que não é encontrado em outras partes do Nepal. O consumo de carne de porco era considerado tabu até algumas décadas atrás. Devido à mistura com a cozinha Kirat do leste do Nepal, a carne de porco encontrou um lugar nos pratos de Katmandu. Uma população marginal de hindus e muçulmanos devotos o considera tabu. Os muçulmanos proíbem comer buff a partir do Alcorão, enquanto os hindus comem todas as variedades, exceto a carne de vaca, pois consideram a vaca uma deusa e símbolo da pureza. O café da manhã principal para moradores e vi

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [26]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [27]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,57274fac5951b619008f8815,Late_Middle_Ages,"A escola de pensamento predominante no século XIII foi a reconciliação tomística dos ensinamentos de Aristóteles com a teologia cristã. A condenação de 1277, promulgada na Universidade de Paris, impôs restrições a idéias que poderiam ser interpretadas como heréticas; restrições que tiveram implicações no pensamento aristotélico. Uma alternativa foi apresentada por Guilherme de Ockham, que insistia que o mundo da razão e o mundo da fé deviam ser mantidos separados. Ockham introduziu o princípio da parcimônia - ou a navalha de Occam -, segundo a qual uma teoria simples é preferida a uma mais complexa, e a especulação sobre fenômenos não observáveis é evitada.",Qual filósofo acreditava que razão e fé deveriam ser mantidas separadas?,"{'answer_start': [367], 'text': ['Guilherme de Ockham']}"
1,572a85fc111d821400f38bb9,Miami,"O sistema rodoviário de Miami é baseado no ""Miami Grid"" numérico, onde a Flagler Street forma a linha de base leste-oeste e a Miami Avenue forma o meridiano norte-sul. A esquina da Flagler Street e Miami Avenue fica no meio do centro da cidade, em frente ao Downtown Macy's (anteriormente a sede do Burdine). A grade de Miami é principalmente numérica, de modo que, por exemplo, todos os endereços ao norte da Flagler Street e a oeste da Miami Avenue têm ""NW"" em seu endereço. Como seu ponto de origem é no Downtown, que fica próximo à costa, os quadrantes ""NW"" e ""SW"" são muito maiores que os quadrantes ""SE"" e ""NE"". Muitas estradas, especialmente as principais, também são nomeadas (por exemplo, Tamiami Trail / SW 8th St), embora, com exceções, o número seja mais comum entre os habitantes locais.",Como é chamado o plano de rua de Miami?,"{'answer_start': [44], 'text': ['Miami Grid']}"
2,572879b54b864d1900164a22,History_of_India,"Ronald Inden escreve que, no século VIII dC, os símbolos dos deuses hindus ""substituíram o Buda no centro imperial e no auge do sistema político-cosmo, a imagem ou o símbolo do deus hindu passa a ser alojado em um templo monumental e é cada vez mais elaborado. adoração ao puja em estilo imperial "". Embora o budismo não tenha desaparecido da Índia por vários séculos após o oitavo, as tendências reais pelos cultos de Vishnu e Shiva enfraqueceram a posição do budismo dentro do contexto sociopolítico e ajudaram a tornar possível seu declínio.",Com que dentadura o budismo desapareceu da maior parte da Índia?,"{'answer_start': [375], 'text': ['oitavo']}"
3,5725dbef271a42140099d2af,Montevideo,"O Museu Nacional de História de Montevidéu está localizado na residência histórica do general Fructuoso Rivera. Ele exibe artefatos relacionados à história do Uruguai. Em um processo iniciado em 1998, o Museu Nacional de História Natural (1837) e o Museu Nacional de Antropologia (1981), fundiram-se em 2001, tornando-se o Museu Nacional de História Natural e Antropologia. Em julho de 2009, as duas instituições tornaram-se novamente independentes. O Museu Histórico anexou oito casas históricas na cidade, cinco das quais estão localizadas na Ciudad Vieja. Um deles, no mesmo quarteirão do edifício principal, é a residência histórica de Antonio Montero, que abriga o Museo Romantico.",Quantas casas o Museu Histórico anexou?,"{'answer_start': [475], 'text': ['oito']}"
4,570e68670dc6ce1900205033,Order_of_the_British_Empire,"A Ordem Mais Excelente do Império Britânico é a ""ordem da cavalaria da monarquia constitucional britânica"", recompensando contribuições para as artes e as ciências, trabalhando com organizações de caridade e bem-estar e serviço público fora do Serviço Civil. Foi criada em 4 de junho de 1917 pelo rei George V e compreende cinco classes, nas divisões civil e militar, das quais as duas mais antigas tornam o beneficiário um cavaleiro, se for homem, ou dame, se for mulher. Há também a Medalha do Império Britânico relacionada, cujos destinatários são afiliados, mas não são membros do pedido.",Quando foi estabe

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## 7. Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [28]:
# from transformers import AutoTokenizer
    
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [29]:
from transformers import AutoTokenizer

if model_checkpoint_original == "neuralmind/bert-large-portuguese-cased":
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_original_local)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [30]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [31]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 4629, 352, 847, 17008, 22282, 12204, 136, 102, 5494, 12204, 847, 16775, 256, 147, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [32]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [33]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Without any truncation, we get the following length for the input IDs:

In [34]:
len(tokenizer(example["question"], example["context"])["input_ids"])

407

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [35]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [36]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several: 

In [37]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 165]

And if we decode them, we can see the overlap:

In [38]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] Em que meses a monção ocorre no Nepal? [SEP] A cidade geralmente tem um clima com dias quentes, seguido de noites e manhãs frias. Prevê - se um clima imprevisível, uma vez que as temperaturas podem cair para 1 ° C ( 34 ° F ) ou menos durante o inverno. Durante uma frente fria de 2013, as temperaturas de inverno de Katmandu caíram para - 4 ° C ( 25 ° F ), e a temperatura mais baixa foi registrada em 10 de janeiro de 2013, a - 9, 2 ° C ( 15, 4 ° F ). As chuvas são principalmente baseadas em monções ( cerca de 65 % do total concentrado durante os meses das monções de Junho a agosto ) e diminuem substancialmente ( 100 a 200 cm ) do leste do Nepal ao oeste do Nepal. As chuvas foram registradas em cerca de 1. 400 milímetros ( 55, 1 polegadas ) no vale de Katmandu e em média 1. 407 milímetros ( 55, 4 polegadas ) na cidade de Katmandu. Em média, a umidade é de 75 %. O gráfico abaixo é baseado em dados do Bureau de Padrões e Meteorologia do Nepal, " Meteorologia Meteorológica " de 2005. O

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [39]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 2), (3, 6), (7, 12), (13, 14), (15, 18), (18, 21), (22, 28), (29, 31), (32, 34), (34, 37), (37, 38), (0, 0), (0, 1), (2, 8), (9, 19), (20, 23), (24, 26), (27, 32), (33, 36), (37, 41), (42, 49), (49, 50), (51, 58), (59, 61), (62, 68), (69, 70), (71, 76), (76, 77), (78, 82), (82, 83), (83, 84), (85, 88), (88, 90), (90, 91), (91, 93), (94, 96), (97, 102), (103, 108), (108, 115), (115, 116), (117, 120), (121, 124), (125, 128), (129, 131), (132, 144), (145, 150), (151, 155), (156, 160), (161, 162), (163, 164), (165, 166), (167, 168), (168, 170), (171, 172), (173, 174), (174, 175), (176, 178), (179, 184), (185, 192), (193, 194), (195, 202), (202, 203), (204, 211), (212, 215), (216, 222), (223, 227), (228, 230), (231, 235), (235, 236), (237, 239), (240, 252), (253, 255), (256, 263), (264, 266), (267, 270), (270, 274), (274, 275), (276, 282), (283, 287), (288, 289), (289, 290), (291, 292), (293, 294), (295, 296), (296, 298), (299, 300), (301, 302), (302, 303), (303, 304), (305, 30

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [40]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

Em Em


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [41]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [42]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

143 145


And we can double check that it is indeed the theoretical answer:

In [43]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

Junho a agosto
Junho a agosto


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [44]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [45]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [46]:
features = prepare_train_features(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## 8. Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [48]:
# from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

if model_checkpoint_original == "neuralmind/bert-large-portuguese-cased":
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint_original_local)
else:
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [108]:
model_num_param=0
for p in model.parameters():
    model_num_param+=p.numel()
model_num_param

333348866

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

## 9. Adapters

In [51]:
if madx2:
    leave_out = [len(model.bert.encoder.layer)-1]
else:
    leave_out = list()

Now we have everything set up to load our AdapterFusion setup. First, we load the (best) lang adapter pre-trained on SQuAD 1.1 pt and the 2 adapters QA and NER respectively pre-trained on SQuAd 1.1 pt and on LeNER-Br. As we don't need their prediction heads, we pass `with_head=False` to the loading method. Next, we add a new fusion layer that combines all the adapters we've just loaded.

In [52]:
from transformers.adapters.composition import Fuse

task_mlm_load_as = 'mlm'
load_as_ad1 = str(task_ad1) + '_' + dataset_name_ad1
load_as_ad2 = str(task_ad2) + '_' + dataset_name_ad2

# Load the pre-trained adapters we want to fuse
if with_adapters_mlm:
    model.load_adapter(load_lang_adapter, config=lang_adapter_config, load_as=task_mlm_load_as, with_head=False)
model.load_adapter(load_qa_adapter, load_as=load_as_ad1, with_head=False)
model.load_adapter(load_ner_adapter, load_as=load_as_ad2, with_head=False)

# Add a fusion layer for QA and NER adapters
model.add_adapter_fusion(Fuse(load_as_ad1, load_as_ad2))

# Put the adapters on the forward ppath
if with_adapters_mlm:
    model.set_active_adapters([task_mlm_load_as, Fuse(load_as_ad1, load_as_ad2)])
else:
    model.set_active_adapters(Fuse(load_as_ad1, load_as_ad2))

# Unfreeze and activate fusion setup
adapter_setup = Fuse(load_as_ad1, load_as_ad2)
model.train_adapter_fusion(adapter_setup)

In [53]:
import torch
from torch import nn

if madx2:
    model.bert.encoder.layer[-1].output.adapter_fusion_layer = nn.ModuleDict(dict())

In [54]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (L

<!-- model_adapter_num_param=0
for p in model.parameters():
    model_adapter_num_param+=p.numel()
model_adapter_num_param

wam = False
411827074 -->

In [55]:
model_adapter_num_param=0
for p in model.parameters():
    model_adapter_num_param+=p.numel()
model_adapter_num_param

411827074

In [56]:
print(f"Number of parameters of the model with adapter: {model_adapter_num_param:.0f}")
print(f"Number of parameters of the model without adapter: {model_num_param:.0f}")
print(f"Number of parameters of the adapter: {model_adapter_num_param - model_num_param:.0f}")
print(f"Pourcentage of additional parameters through adapter:",round(((model_adapter_num_param - model_num_param)/model_num_param)*100,2),'%')

Number of parameters of the model with adapter: 411827074
Number of parameters of the model without adapter: 333348866
Number of parameters of the adapter: 78478208
Pourcentage of additional parameters through adapter: 23.54 %


## 10. Training

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [54]:
from transformers import TrainingArguments

if ds:
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        do_train=do_train,
        do_eval=do_eval,
        evaluation_strategy=evaluation_strategy,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        num_train_epochs=num_train_epochs,
        lr_scheduler_type=lr_scheduler_type,
        warmup_ratio=warmup_ratio,
        warmup_steps=warmup_steps,
        logging_dir=logging_dir,         # directory for storing logs
        logging_strategy=evaluation_strategy,
        logging_steps=logging_steps,     # if strategy = "steps"
        save_strategy=evaluation_strategy,          # model checkpoint saving strategy
        save_steps=logging_steps,        # if strategy = "steps"
        save_total_limit=save_total_limit,
        fp16=fp16,
        eval_steps=logging_steps,        # if strategy = "steps"
        load_best_model_at_end=load_best_model_at_end,
        metric_for_best_model=metric_for_best_model,
        greater_is_better=greater_is_better,
        local_rank=gpu,
        deepspeed = ds_config
        )
else:
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        do_train=do_train,
        do_eval=do_eval,
        evaluation_strategy=evaluation_strategy,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        num_train_epochs=num_train_epochs,
        lr_scheduler_type=lr_scheduler_type,
        warmup_ratio=warmup_ratio,
        warmup_steps=warmup_steps,
        logging_dir=logging_dir,         # directory for storing logs
        logging_strategy=evaluation_strategy,
        logging_steps=logging_steps,     # if strategy = "steps"
        save_strategy=evaluation_strategy,          # model checkpoint saving strategy
        save_steps=logging_steps,        # if strategy = "steps"
        save_total_limit=save_total_limit,
        fp16=fp16,
        eval_steps=logging_steps,        # if strategy = "steps"
        load_best_model_at_end=load_best_model_at_end,
        metric_for_best_model=metric_for_best_model,
        greater_is_better=greater_is_better,
        local_rank=gpu,
        )

And second, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [55]:
from transformers import default_data_collator

data_collator = default_data_collator

Then we just have to pass everything to `Trainer` and begin training:

In [56]:
from transformers.trainer_callback import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
#         do_save_full_model=train_adapter, # save full model as we finetuned head + embeddings
    do_save_full_model=not train_adapter, # save full model as we finetuned head + embeddings
    do_save_adapters=train_adapter,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
    )    

In [ ]:
outputs

In [ ]:
trainer.train()

In [59]:
# check weights dtype
trainer.model.bert.embeddings.word_embeddings.weight.dtype

torch.float32

In [61]:
# save adapter + head
adapters_folder = 'adapters-' + task_ad1
path_to_save_adapter = path_to_outputs/adapters_folder
load_as_fusion = load_as_ad1 + ',' + load_as_ad2

trainer.model.save_adapter_fusion(path_to_save_adapter, load_as_fusion)
trainer.model.save_all_adapters(path_to_save_adapter)
trainer.model.save_head(path_to_save_adapter)

!ls -lh {path_to_save_adapter}

total 277M
-rw-rw-r-- 1 pierre pierre  406 Aug  1 05:44 adapter_fusion_config.json
-rw-rw-r-- 1 pierre pierre  225 Aug  1 05:44 head_config.json
drwxrwxr-x 2 pierre pierre 4.0K Aug  1 05:44 ner_lenerbr
-rw-rw-r-- 1 pierre pierre 277M Aug  1 05:44 pytorch_model_adapter_fusion.bin
-rw-rw-r-- 1 pierre pierre 9.0K Aug  1 05:44 pytorch_model_head.bin
drwxrwxr-x 2 pierre pierre 4.0K Aug  1 05:44 qa_squad11pt


Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
import math
eval_results = trainer.evaluate()
eval_results

In [ ]:
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

## 11. TensorBoard

In [64]:
#!pip install tensorboard

In [ ]:
import os
PATH = os.getenv('PATH')
# repalce xxxx by username
# replace yyyy by virtual ambiente name
%env PATH=/mnt/home/xxxx/anaconda3/envs/yyyy/bin:$PATH

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {logging_dir} --bind_all

## 12. Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [68]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [69]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [70]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 35,  80, 105,  77, 183,   0,  22,   0,  40, 129,  47,  53,  53,  39,
          39,  76], device='cuda:1'),
 tensor([ 38,  81, 107,  80, 184,   0,  22,   0,  42, 149,  52,  60,  60,  51,
          51, 105], device='cuda:1'))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [71]:
n_best_size = 20

In [72]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [73]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [75]:
raw_predictions = trainer.predict(validation_features)

The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [76]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [77]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

We can compare to the actual ground-truth answer:

In [ ]:
datasets["validation"][0]["answers"]

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [80]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [81]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [82]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10917 features.


Then we can load the metric from the datasets library.

In [83]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

## 13. Application QA

In [85]:
### import transformers
import pathlib
from pathlib import Path

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

if model_checkpoint_original == "neuralmind/bert-large-portuguese-cased":
    tokenizer_qa = AutoTokenizer.from_pretrained(model_checkpoint_original_local)
    model_qa = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint_original_local)
else:
    tokenizer_qa = AutoTokenizer.from_pretrained(model_checkpoint)
    model_qa = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

### Lang adapter

In [87]:
if with_adapters_mlm:
    
    # hyperparameters used for fine-tuning the MLM with lang adapter
    learning_rate_mlm = 1e-4
    batch_size_mlm = 16
    gradient_accumulation_steps_mlm = 1
    adam_epsilon_mlm = 1e-4
    num_train_epoch_mlm = 100.
    early_stopping_patience_mlm = 10
    madx2_mlm = madx2
    houlsby_MHA_lastlayer_mlm = houlsby_MHA_lastlayer
    ds_mlm = False
    fp16_mlm = True
    load_best_model_at_end_mlm = True
    metric_for_best_model_mlm = "loss"
    adapter_config_mlm = adapter_config_name + '+inv'

    # path to lang adapter
    outputs_mlm = model_checkpoint.replace('/','-') + '_' + dataset_name + '/' + 'mlm' + '/' \
    + 'lr' + str(learning_rate_mlm) \
    + '_bs' + str(batch_size_mlm) \
    + '_GAS' + str(gradient_accumulation_steps_mlm) \
    + '_eps' + str(adam_epsilon_mlm) \
    + '_epochs' + str(num_train_epoch_mlm) \
    + '_patience' + str(early_stopping_patience_mlm) \
    + '_madx2' + str(madx2_mlm) \
    + '_houlsby_MHA_lastlayer' + str(houlsby_MHA_lastlayer_mlm) \
    + '_ds' + str(ds_mlm) \
    + '_fp16' + str(fp16_mlm) \
    + '_best' + str(load_best_model_at_end_mlm) \
    + '_metric' + str(metric_for_best_model_mlm) \
    + '_adapterconfig' + str(adapter_config_mlm)

    path_to_outputs_mlm = root/'outputs'/outputs_mlm

    # Config of the lang adapter
    lang_adapter_path = path_to_outputs_mlm/'adapters-mlm/'
    
    load_lang_adapter = str(lang_adapter_path)
    lang_adapter_config = str(lang_adapter_path) + "/adapter_config.json"
    
else:
    type_lang_adapter = None

    load_lang_adapter = None
    lang_adapter_config = None

In [88]:
if with_adapters_mlm:
    # load the language adapter without head
    task_mlm_load_as = 'mlm'
    lang_adapter_name = model_qa.load_adapter(
        load_lang_adapter,
        config=lang_adapter_config,
        load_as=task_mlm_load_as,
        with_head=False
        )

### Fusion adapter

In [89]:
# load adapter
adapter_name_ad1 = model_qa.load_adapter(
    load_qa_adapter,
    config=qa_adapter_config,
    load_as=load_as_ad1, 
    with_head=False
)
    
adapter_name_ad2 = model_qa.load_adapter(
    load_ner_adapter,
    config=ner_adapter_config,
    load_as=load_as_ad2, 
    with_head=False
)
    
model_qa.load_adapter_fusion(path_to_save_adapter)
    
model_qa.load_head(path_to_save_adapter)

adapter_setup = [
    [
        load_as_ad1,
        load_as_ad2
    ]
]

### Active adapter

In [90]:
# Set the adapters to be used in every forward pass
if with_adapters_mlm:
    model_qa.set_active_adapters([lang_adapter_name,adapter_setup[0]]) 
else:
    model_qa.set_active_adapters(adapter_setup[0])    

In [91]:
model_qa

BertForQuestionAnswering(
  (bert): BertModel(
    (invertible_adapters): ModuleDict(
      (mlm): NICECouplingBlock(
        (F): Sequential(
          (0): Linear(in_features=512, out_features=256, bias=True)
          (1): Activation_Function_Class()
          (2): Linear(in_features=256, out_features=512, bias=True)
        )
        (G): Sequential(
          (0): Linear(in_features=512, out_features=256, bias=True)
          (1): Activation_Function_Class()
          (2): Linear(in_features=256, out_features=512, bias=True)
        )
      )
    )
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(

In [92]:
from transformers import pipeline
nlp = pipeline("question-answering", model=model_qa, tokenizer=tokenizer_qa)

In [93]:
# source: https://pt.wikipedia.org/wiki/Pandemia_de_COVID-19
context = r"""A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
uma doença respiratória causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
O vírus tem origem zoonótica e o primeiro caso conhecido da doença remonta a dezembro de 2019 em Wuhan, na China. 
Em 20 de janeiro de 2020, a Organização Mundial da Saúde (OMS) classificou o surto 
como Emergência de Saúde Pública de Âmbito Internacional e, em 11 de março de 2020, como pandemia. 
Em 18 de junho de 2021, 177 349 274 casos foram confirmados em 192 países e territórios, 
com 3 840 181 mortes atribuídas à doença, tornando-se uma das pandemias mais mortais da história.
Os sintomas de COVID-19 são altamente variáveis, variando de nenhum a doenças com risco de morte. 
O vírus se espalha principalmente pelo ar quando as pessoas estão perto umas das outras. 
Ele deixa uma pessoa infectada quando ela respira, tosse, espirra ou fala e entra em outra pessoa pela boca, nariz ou olhos.
Ele também pode se espalhar através de superfícies contaminadas. 
As pessoas permanecem contagiosas por até duas semanas e podem espalhar o vírus mesmo se forem assintomáticas.
"""

In [ ]:
%%time
question = "Quando começou a pandemia de Covid-19 no mundo?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Qual é a data de início da pandemia Covid-19 em todo o mundo?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "A Covid-19 tem algo a ver com animais?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Onde foi descoberta a Covid-19?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quantos casos houve?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quantos mortes?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quantos paises tiveram casos?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Quais são sintomas de COVID-19"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
%%time
question = "Como se espalha o vírus?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

# END